In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from math import floor

In [2]:
total_csvs = !ls total_*

In [3]:
def write_excel(df: pd.DataFrame, file_name: str, sheet: str = 'Sheet1'):
    writer = pd.ExcelWriter(f'{file_name}.xlsx')
    df.to_excel(writer, sheet)
    writer.save()

def read_csv(file_name: str, **args: dict) -> pd.DataFrame:
    return pd.read_csv(filepath_or_buffer=file_name, **args)


def file_name_suffix(name: str) -> str:
    return name[name.rfind('_') + 1:-4]

total_names = ['fuel', 'co2']
df_totals = [(file_name_suffix(name),
              read_csv(
                  file_name=name,
                  encoding='utf-8',
                  names=total_names,
                  header=0)) for name in total_csvs]
df_new_co2 = read_csv(
    file_name='fordonspark.csv',
    encoding='utf-8',
    header=0,
    names=['license_nbr', 'co2'])
df_cars = pd.read_csv(
    'cars.csv',
    encoding='latin1',
    sep=';',
    header=0,
    names=[
        'license_nbr', 'brand', 'year', 'driver', 'region', 'consumption',
        'co2', 'fuel'
    ])

In [4]:
df_cars['brand'] = df_cars['brand'].apply(lambda b: ' '.join(b.upper().split(' ')[:2]))
df_cars['co2'] = df_new_co2['co2']
df_cars.head()

,license_nbr,brand,year,driver,region,consumption,co2,fuel
0,YZF 336,VW TRANSPORTER,2018,ALBIN LÖFSTRAND,ÖST,"0,62",36464,Diesel
1,PYN 113,MB SPRINTER,2018,ALEXANDER WERNERSSON,SYD,"0,81",45974,Diesel
2,JUB 435,VW TRANSPORTER,2016,ANDERS ANDERSSON,SYD,"0,77",44114,Diesel
3,YLJ 847,MB SPRINTER,2017,ANDERS BONNIER,SYD,"0,81",45974,Diesel
4,YHN 351,AUDI Q5,2017,ANDERS KARLSSON,ÖST,"0,59",18098,Diesel


In [10]:
print(df_totals[0][0])
df_totals[0][1].head()

big


,fuel,co2
0,Konventionell diesel,45530
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),37334
2,Gasfordon,26009


In [9]:
print(df_totals[1][0])
df_totals[1][1].head()

medium


,fuel,co2
0,Konventionell diesel,36418
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),32744
2,Konventionell bensin,37787
3,Biobensin E4.8 (4.8% bioinblandning),34988


In [11]:
print(df_totals[2][0])
df_totals[2][1].head()

small


,fuel,co2
0,Konventionell diesel,27742
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),23182
2,Konventionell bensin,32822
3,Biobensin E4.8 (4.8% bioinblandning),29587
4,Elfordon skåp. 26.7 kWh,13543


In [23]:
# Find best fuel value for every category
def find_opt_co2(totals):
    cat_co2_dict = {}
    co2_fuel_dict = {}
    for cat, df in totals:
        min_idx = df['co2'].idxmin()
        fuel, co2 = df.iloc[min_idx, 0], df.iloc[min_idx, 1]
        cat_co2_dict[cat] = co2
        co2_fuel_dict[co2] = fuel
    return cat_co2_dict, co2_fuel_dict


cat_co2_dict, co2_fuel_dict = find_opt_co2(df_totals)

In [24]:
car_cats = {
    'service': {
        'small': set(['VW CADDY']),
        'medium': set(['MB VITO', 'VW TRANSPORTER']),
        'big': set(['MB SPRINTER', 'VW TRANSPORT'])
    },
    'work':
    set([
        'AUDI A6', 'AUDI Q5', 'BMW 220D', 'BMW 318D', 'BMW 320D',
        'SKODA SUPERB', 'VOLVO S60', 'VOLVO S90', 'VOLVO V60', 'VOLVO V90',
        'VOLVO XC40', 'VOLVO XC60', 'VOLVO XC70', 'VW PASSAT', 'VW TIGUAN',
        'VW TOUAREG'
    ])
}

service_brands = set(chain(*car_cats['service'].values()))
work_brands = car_cats['work']

def get_car_cat(brand):
    if brand in work_brands:
        return 'work'
    for k, v in car_cats['service'].items():
        if brand in v:
            return k
    raise Exception(f'Could not classify brand {brand}')

In [25]:
# Assign new column 'category' and filter cars
df_cars = df_cars.assign(category=[get_car_cat(brand) for brand in df_cars['brand']])
df_filtered = df_cars.query("(brand in @service_brands & year < 2015) | (brand in @work_brands & year < 2017)")

In [26]:
# Treat 25% of big service cars as medium
sorted_big = df_filtered.query("category == 'big' & (region == 'ÖST' | region == 'SYD' | region == 'VÄST')").sort_values('co2', ascending=False)
top_25_big = sorted_big.iloc[:floor(len(sorted_big) * 0.25)]
df_replaced_big = df_filtered.copy()
df_replaced_big.loc[top_25_big.index.values.tolist(), 'category'] = 'medium'

In [27]:
# Assign new column 'new_co2'
df_replaced_big = df_replaced_big.assign(new_co2=[cat_co2_dict[cat] for cat in df_filtered['category']])

In [28]:
df_replaced_big.loc[top_25_big.index.values.tolist(), 'category'] = 'big' # Show the 25% cars as big again
df_scen1 = df_cars.copy()
df_scen1['new_co2'] = np.nan
df_scen1.update(df_replaced_big)
df_scen1 = df_scen1.assign(new_fuel=[co2_fuel_dict.get(co2, np.nan) for co2 in df_scen1['new_co2']])

In [29]:
df_scen1

,license_nbr,brand,year,driver,region,consumption,co2,fuel,category,new_co2,new_fuel
0,YZF 336,VW TRANSPORTER,2018.0,ALBIN LÖFSTRAND,ÖST,"0,62",36464.0,Diesel,medium,NaN,NaN
1,PYN 113,MB SPRINTER,2018.0,ALEXANDER WERNERSSON,SYD,"0,81",45974.0,Diesel,big,NaN,NaN
2,JUB 435,VW TRANSPORTER,2016.0,ANDERS ANDERSSON,SYD,"0,77",44114.0,Diesel,medium,NaN,NaN
3,YLJ 847,MB SPRINTER,2017.0,ANDERS BONNIER,SYD,"0,81",45974.0,Diesel,big,NaN,NaN
4,YHN 351,AUDI Q5,2017.0,ANDERS KARLSSON,ÖST,"0,59",18098.0,Diesel,work,NaN,NaN
5,BAP 210,VW CADDY,2017.0,ANDERS NILSSON,SYD,"0,50",31034.0,Diesel,small,NaN,NaN
6,RSA 761,VW TRANSPORT,2015.0,ANDREAS ENGSTRÖM,ÖST,"0,84",47444.0,Diesel,big,NaN,NaN
7,YHR 669,BMW 320D,2016.0,ANDREAS FRITZON,MITT,"0,47",15866.0,Diesel,work,10126.0,Fordonsgas
8,EFM 275,BMW 320D,2017.0,ANDREAS KNUTSEN-ÖY,MITT,"0,44",15278.0,Diesel,work,NaN,NaN
9,ELW 214,VW TRANSPORT,2016.0,ANDREAS LÖÖV,MITT,"0,84",47444.0,Diesel,big,NaN,NaN


In [18]:
write_excel(df_scen1, 'scen1')